In [1]:
# ============================================================
# 04_interpretation.ipynb — Interpret Results + Insights
# Goal: Turn evaluation + error analysis into clear findings, limitations, and next steps for README/portfolio.
# ============================================================

In [2]:
# --------------------------------------------
# Imports + project paths
# --------------------------------------------
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..")  # notebooks/ -> project root
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
FIGURES_DIR = PROJECT_ROOT / "figures"

# Inline sanity check (helps catch path issues early)
print("DATA_PROCESSED:", DATA_PROCESSED.resolve())
print("FIGURES_DIR:", FIGURES_DIR.resolve())

DATA_PROCESSED: /Users/saam/PycharmProjects/ev-range-prediction/data/processed
FIGURES_DIR: /Users/saam/PycharmProjects/ev-range-prediction/figures


In [3]:
# --------------------------------------------
# Load artifacts produced in 03 (or earlier)
# --------------------------------------------
pred_path = DATA_PROCESSED / "pred_df_test.csv"
worst_path = DATA_PROCESSED / "worst_errors.csv"

pred_df = pd.read_csv(pred_path)
worst_errors = pd.read_csv(worst_path)

print("pred_df shape:", pred_df.shape)
print("worst_errors shape:", worst_errors.shape)
pred_df.head()

pred_df shape: (262, 6)
worst_errors shape: (10, 6)


,make,model,year,y_true,y_pred,abs_error
0,GMC,Hummer EV Pickup 2M20,2024,311,297.143333,13.856667
1,Vinfast,VF 8 Plus,2025,235,245.623333,10.623333
2,Rivian,R1T Performance Dual Large (22in),2024,341,341.333333,0.333333
3,Toyota,bZ4X,2024,252,246.756667,5.243333
4,Rivian,R1S Quad Large (20in),2024,289,291.833333,2.833333


In [4]:
# --------------------------------------------
# Compute key test metrics from pred_df
# --------------------------------------------
import numpy as np

y_true = pred_df["y_true"].to_numpy()
y_pred = pred_df["y_pred"].to_numpy()

mae = np.mean(np.abs(y_true - y_pred))
rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

# R^2 from definition (no sklearn needed)
ss_res = np.sum((y_true - y_pred) ** 2)
ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
r2 = 1 - (ss_res / ss_tot)

summary = pd.DataFrame([{
    "MAE (miles)": mae,
    "RMSE (miles)": rmse,
    "R²": r2
}])

summary

,MAE (miles),RMSE (miles),R²
0,16.069504,23.320036,0.897312


In [5]:
# --------------------------------------------
# Interpretation notes
# --------------------------------------------
notes = [
    f"Test MAE is ~{mae:.1f} miles, meaning predictions are typically off by about that many miles.",
    f"Test RMSE is ~{rmse:.1f} miles, which penalizes the largest misses more heavily than MAE.",
    f"Test R² is ~{r2:.3f}, indicating how much variance in EV range the model explains on unseen data.",
    "Error plots in /figures help identify where the model is strong vs weak (brands/years/outliers)."
]

for n in notes:
    print("- " + n)

- Test MAE is ~16.1 miles, meaning predictions are typically off by about that many miles.
- Test RMSE is ~23.3 miles, which penalizes the largest misses more heavily than MAE.
- Test R² is ~0.897, indicating how much variance in EV range the model explains on unseen data.
- Error plots in /figures help identify where the model is strong vs weak (brands/years/outliers).


In [6]:
# --------------------------------------------
# Reference saved figures (file names should exist in /figures)
# --------------------------------------------
figure_files = [
    "pred_vs_actual_test.png",
    "abs_error_distribution.png",
    "mae_by_brand.png",
    "mae_by_year.png",
    "mae_by_support_level.png",
]

missing = [f for f in figure_files if not (FIGURES_DIR / f).exists()]
print("Missing figures:", missing)


Missing figures: []


In [7]:
# --------------------------------------------
# Export interpretation summary for README
# --------------------------------------------
out_md = DATA_PROCESSED / "interpretation_summary.md"

md = f"""# EV Range Prediction — Interpretation Summary

## Test-set performance
- **MAE:** {mae:.2f} miles
- **RMSE:** {rmse:.2f} miles
- **R²:** {r2:.4f}

## What the plots suggest
- Predicted vs Actual: points close to the diagonal indicate good fit; outliers show worst misses.
- Absolute error distribution: shows typical error and tail risk (rare large misses).
- MAE by brand/year: highlights segments where the model generalizes poorly (often data scarcity or unusual models).

## Limitations (important for credibility)
- Some models/brands have low support → higher uncertainty.
- EPA range can differ from real-world range; model predicts the dataset’s target definition.

## Next steps
- Add more features (battery kWh, weight, efficiency) if available.
- Try a stronger model / tuning, or calibrate predictions for high-range vehicles.
"""

out_md.write_text(md, encoding="utf-8")
print("Saved:", out_md.resolve())

Saved: /Users/saam/PycharmProjects/ev-range-prediction/data/processed/interpretation_summary.md
